# Рекомендация тарифов

## Описание проекта
Оператор мобильной связи `Мегалайн` выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «`Смарт`» или «`Ультра`».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «`Статистический анализ данных`»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

## Цель выполнения проекта
Построение модели для задачи классификации (выбор подходящего тарифа мобильной связи) с максимально большим значением `accuracy`.

## Критерии успешной реализации проекта
Построение модели с максимально большим значением `accuracy`. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до `0.75`. 

Выполнить проверку `accuracy` на тестовой выборке самостоятельно.

Критерии оценки проекта, которыми руководствуются ревьюверы:

- как вы изучаете данные после загрузки?
- корректно ли разделяете данные на выборки?
- как выбираете размеры выборок?
- правильно ли вы оцениваете качество моделей в исследовании?
- какие модели и гиперпараметры вы используете?
- какие выводы об исследовании делаете?
- правильно ли тестируете модели?
- насколько высокое значение accuracy получаете?
- соблюдаете структуру проекта и поддерживаете аккуратность кода?

## Задачи проекта
В рамках проекта решается ряд задач, выполнение которых способствует достижению цели проекта:

1. [открытие файла с данными и изучение его](#Изучение-общей-информации)
    * загрузка библиотек
    * открытие датасета
    * определение целевого параметра
2. [разделение исходных данных на обучающую, валидационную и тестовую выборки](#Разделение-исходных-данных-на-выборки)
3. [исследование качества разных моделей, путём изменения гиперпараметров](#Исследование-модели)
    * исследование модели классификации деревом решений
    * исследование модели классификации - случайный лес
    * исследование модели классификации - логическая регрессия
    * краткие выводы исследования
4. [проверка качества модели на тестовой выборке](#Проверка-модели-на-тестовой-выборке)
5. [дополнительное задание: проверьте модели на вменяемость](#Проверка-модели-на-адекватность-(бонус)) 
6. [заполнение чек-листа готовности проекта](#Чек-лист-готовности-проекта)

## Исходные данные
Откройте файл с данными и изучите его. Путь к файлу: `/datasets/users_behavior.csv`.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

В датасете присутствуют следующие параметры:

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — `1`, «Смарт» — `0`).

## Выполнение проекта

### Изучение общей информации

Откройте файл с данными и изучите его. Путь к файлу `/datasets/users_behavior.csv`

#### Загрузка библиотек

In [85]:
import pandas as pd                                     # импорт библиотеки pandas

from sklearn.linear_model import LogisticRegression     # импорт модели LogisticRegression из модуля sklearn.linear_model,
                                                        # реализующий алгоритм дерева решений для задач классификации

from sklearn.tree import DecisionTreeClassifier         # импорт функции DecisionTreeClassifier из модуля sklearn.tree

from sklearn.ensemble import RandomForestClassifier     # импорт функции RandomForestClassifier из модуля sklearn.ensemble

from sklearn.model_selection import train_test_split    # импорт функции train_test_split для разделения исходных данных на выборки


from joblib import dump                                 # импорт функции dump из библиотеки joblib, предназначенный 
                                                        # в т.ч. для загрузки результатов полученных вычислений

from sklearn.metrics import accuracy_score              # импорт функции accuracy_score для расчёта метрики качества

#from sklearn.metrics import mean_squared_error          # импорт функции mean_squared_error 
                                                        # для расчёта среднеквадратичного отклонения из модуля sklearn.metrics

#### Открытие датасета

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.shape

(3214, 5)

#### Определение целевого параметра


Перед нами стоит цель выбора подходящего тарифа мобильной связи. Собственно говоря, в рассматриваемом датасете уже есть параметр `is_ultra`. Данный параметр категориальный, ввиду чего принимаем решение, что решаться будет задача классификации.

In [4]:
features = data.drop('is_ultra', axis=1)       # объявление переменной признаков
target = data['is_ultra']                      # объявление целевого признака

### Разделение исходных данных на выборки

В нашем распоряжении находится датасет, из которого необходимо сделать разделение на три выборки: обучающую (тренировочную), валидационную (на ней будет проверяться качество обученной модели) и тестовую. Подобная постановка задачи чётко указывает на отсутствие даже намёка на существование тестовой выборки. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1. 

Вначале разбиваем данные на обучающую (в названии фигурирует `_train`) и валидационно-тестовую (в названии `_validtest`) выборки в соотношении 3:2 (60% необходимо отвести под обучение модели), а затем уже делим валидационно-тестовую выборку на валидационную (`_valid`) и тестовую (`_test`) выборки в соотношении 1:1. 

In [5]:
features_train, features_validtest, target_train, target_validtest = train_test_split(
    features,
    target,
    test_size=0.4, 
    random_state=12345
)

In [6]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_validtest, 
    target_validtest, 
    test_size=0.5, 
    random_state=12345
)

Выше подсчитано `data.shape`. Если просуммировать значения строк получившихся выборок, то видно, что ничего не потеряно. И действительно исходные данные разделены в заложенных пропорциях.

In [7]:
features_train.shape

(1928, 4)

In [8]:
features_valid.shape

(643, 4)

In [9]:
features_test.shape

(643, 4)

### Исследование модели

Рассмотрим три известные нам модели обучения для решения задачи классификации. _Логистическая регрессия_ подходит для задач с двумя классами, когда данные линейно разделимы. Модель _дерева решений_ подходит для задач, в которых признаки имеют нелинейную зависимость и когда нужно интерпретировать результаты. _Случайный лес_ подходит для задач с большим количеством признаков и когда нужно уменьшить переобучение.

Самое высокое качество accurancy среди представленных моделей у _случайного леса_, а _логистическая регрессия_ на втором месте. При этом _случайный лес_ является самой медленной моделью, что логично, если учесть, для каких целей он используется. Также крайне важно, что модель _случайного леса_ избыточна для нашего датасета. Конечно можно и на БелАЗе пассажиров возить, только зачем, если есть легковые авто.

В нашем случае в обучающей выборке присутствуют всего 4 признака. Для достижения поставленной цели - выбора подходящего тарифа мобильной связи - оптимальнее выбрать модель _логистической регрессии_, т.к. у нас всего два класса (0 и 1) и данные линейно разделимы. Кроме того, _логистическая регрессия_ хорошо работает с небольшим количеством признаков, что соответствует нашему датасету, в отличие от модели _случайного леса_. Также _логистическая регрессия_ позволит нам интерпретировать результаты и понять, какие признаки больше всего влияют на целевой признак `is_ultra`. Несмотря на перечисленные факторы, в рамках проекта также будет опробована модель _дерева решений_ в силу своей применимости для небольших выборок. Одним из факторов, которые не позволяют нам обратиться к модели _дерева решений_ обратиться в первую очередь, является возможность наличия проблемы пере/недообучения. Посмотрим, как в итоге сложится.

#### Исследование модели логистической регрессии

Выполним обучение модели на тренировочной выборке и посчитаем `accurancy`, используя гиперпараметры по-умолчанию

In [10]:
# создание объекта модели логистической регрессии без явного указания заданных гиперпараметров
model_logreg = LogisticRegression()

# обучение модели 
model_logreg.fit(features_train, target_train)

# определение метрики качества accuracy на тренировочной выборке в случае логистической регрессии
result_train_logreg = model_logreg.score(features_train, target_train)

print(f'Величина метрики качества accurancy на валидационной выборке {result_train_logreg}')

# предсказания на валидационной выборке
# predictions_valid = model_logreg.predict(features_valid)

# определение метрики качества accuracy на валидационной выборке в случае логистической регрессии
result_valid_logreg = model_logreg.score(features_valid, target_valid)

print(f'Величина метрики качества accurancy на валидационной выборке {result_valid_logreg}')

Величина метрики качества accurancy на валидационной выборке 0.7131742738589212
Величина метрики качества accurancy на валидационной выборке 0.7107309486780715


Как видно, результат ни в одном из случаев (главным образом, на обучающей выборке) не достиг необходимой величины в 75% правильных ответов. Используем изменение значений гиперпараметров.

##### Использование трёх значений гиперпараметров

Исходя из [официальной документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) на библиотеку `scikit-learn`, для модели _LogisticRegression_ существует ряд параметров, из которых наиболее важными являются следующие:

- `random_state` - начальное значение для генератора случайных чисел
- `solver` - алгоритм оптимизации (newton-cg, lbfgs, liblinear, sag или saga)
- `max_iter` - максимальное количество итераций для оптимизации
- `penalty` - тип регуляризации (l1, l2, elasticnet или none)
- `C` - обратная сила регуляризации (чем больше значение C, тем меньше регуляризация)
- `tol` - критерий остановки оптимизации
- `class_weight` - веса классов для учета дисбаланса классов

Из данного перечня применим параметры `random_state`, `solver` и `max_iter`

In [11]:
for solver_type in ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']:
    try:
        model_logreg = LogisticRegression(random_state=12345, solver=solver_type, max_iter=100000)
        model_logreg.fit(features_train, target_train)
        result_train_logreg = model_logreg.score(features_train, target_train)
        print(f'Величина метрики качества accurancy при значении гиперпараметра solver={solver_type} на валидационной выборке равна: {result_train_logreg}') 
    except ValueError:
        print(f'Алгоритм линейного поиска не смог найти оптимальное значение параметра accurancy для solver={solver_type}') 

Величина метрики качества accurancy при значении гиперпараметра solver=liblinear на валидационной выборке равна: 0.7157676348547718
Величина метрики качества accurancy при значении гиперпараметра solver=lbfgs на валидационной выборке равна: 0.7131742738589212
Величина метрики качества accurancy при значении гиперпараметра solver=newton-cg на валидационной выборке равна: 0.7531120331950207


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorith

Величина метрики качества accurancy при значении гиперпараметра solver=sag на валидационной выборке равна: 0.7105809128630706
Величина метрики качества accurancy при значении гиперпараметра solver=saga на валидационной выборке равна: 0.7085062240663901


Обращает на себя внимание, что при использовании алгоритма оптимизации `newton-sg` удаётся достигнуть необходимого значения правильных ответов обучающейся модели. Ввиду этого добавим гиперпараметры `tol` и `penalty`, подобрав для них подходящие значения

In [12]:
model_logreg = LogisticRegression(random_state=12345, solver='newton-cg', max_iter=100000, tol=0.001, penalty='none')
model_logreg.fit(features_train, target_train)
result_train_logreg = model_logreg.score(features_train, target_train)
print(f'Величина метрики качества accurancy на валидационной выборке равна: {result_train_logreg}') 

Величина метрики качества accurancy на валидационной выборке равна: 0.7531120331950207


На обучающейся выборке достигнуто требуемое значение. Подсчитаем его для валидационной выборки. Как видно, значения близки

In [13]:
result_valid_logreg = model_logreg.score(features_valid, target_valid)
print(f'Величина метрики качества accurancy на валидационной выборке {result_valid_logreg}')

dump(model_logreg, 'model_9_1.joblib')

Величина метрики качества accurancy на валидационной выборке 0.7558320373250389


['model_9_1.joblib']

##### Выводы по результатам исследования

Несмотря на то, что наиболее распространённым является алгоритм `lbfgs` гиперпараметра `solver`, для нашей модели наилучший результат обучения дало применение алгоритма `newton-cg`. Они оба относятся к оптимизационным алгоритмам, которые используются для обучения моделей ML, а отличаются способом вычисления градиента и гессиана функции потерь.

Алгоритм lbfgs (Limited-memory Broyden-Fletcher-Goldfarb-Shanno) использует ограниченную память для хранения предыдущих значений градиента и использует их для приближенного вычисления гессиана. Он является быстрее и менее требовательным к памяти, чем алгоритм newton-cg, но может иметь проблемы с сходимостью на некоторых задачах. Алгоритм newton-cg (Newton's conjugate gradient) использует точное вычисление градиента и гессиана функции потерь, что делает его более точным, но также более медленным и требовательным к памяти. Однако на нашем довольно небольшой выборке (даже изначальный датасет составлял всего 3214 объекта) различия в скоростях вычислений для указанных алгоритмов практически не заметны.

Ввиду того, что алгоритм `newton-cg` показал удовлетворяющие условию результаты в обучаемой модели, то есть смысл его использовать далее. Однако, стоит учитывать, что помимо выбора оптимизационного алгоритма потребовалась также дополнительная настройка гиперпараметров, таких как `tol` и `penalty`.

#### Проверка модели на тестовой выборке

In [14]:
predictions_train_logreg = model_logreg.predict(features_train)
predictions_valid_logreg = model_logreg.predict(features_valid)
predictions_test_logreg = model_logreg.predict(features_test)

In [15]:
print('Значения Accurancy на различных выборках:')
print("Значения Accurancy на обучающейся выборке равно", model_logreg.score(features_train, target_train))
print("Значения Accurancy на валидационной выборке равно", model_logreg.score(features_valid, target_valid))
print("Значения Accurancy на тестовой выборке равно", model_logreg.score(features_test, target_test))

Значения Accurancy на различных выборках:
Значения Accurancy на обучающейся выборке равно 0.7531120331950207
Значения Accurancy на валидационной выборке равно 0.7558320373250389
Значения Accurancy на тестовой выборке равно 0.7387247278382582


Модель логистической регрессии показывает примерно одинаковую точность на обучающей, валидационной и тестовой выборках, что может говорить о ее стабильности и непереобученности. Однако, точность модели на всех выборках не очень высока и составляет около 0.75. Необходимо провести дополнительный анализ и оптимизацию модели для улучшения ее качества.

Для улучшения качества модели логистической регрессии можно попробовать следующие методы:

1. Подбор гиперпараметров: можно использовать методы перебора гиперпараметров, например, [GridSearchCV](https://vc.ru/ml/147132-kak-avtomaticheski-podobrat-parametry-dlya-modeli-mashinnogo-obucheniya-ispolzuem-gridsearchcv) или RandomizedSearchCV, чтобы определить оптимальные значения гиперпараметров модели. Например, можно изменять значения параметров C, tol, max_iter, penalty и solver.

2. Использование других моделей: если точность модели логистической регрессии не удовлетворяет требованиям, можно попробовать использовать другие модели машинного обучения, такие как `случайный лес` или `дерево решений`.

3. Балансировка классов: если классы в целевой переменной несбалансированы, то можно использовать методы балансировки классов, такие как oversampling или undersampling, чтобы улучшить качество модели (это было нагуглено, но пока подобное реализовать не готов).

#### Перебор гиперпараметров GridSearchCV

Перебор гиперпараметров **GridSearchCV** приводит к ряду сообщений и крайне длительной работе. Ввиду чего тут представлены количественные гиперпараметры, которые в итоге были безошибочно проанализированы и на их основе сформирован наиболее оптимальный перечень.

In [16]:
from sklearn.model_selection import GridSearchCV
clf = LogisticRegression()

parametrs = { 
    'random_state': [None, 12345, 777, 54321],
    'max_iter': [1000, 10000, 100000, 1000000],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'tol': [0.001, 0.01, 0.1, 1, 10, 100]
            }
grid = GridSearchCV(clf, parametrs, cv=5)
grid.fit(features_train, target_train)
grid.best_params_

{'C': 0.001, 'max_iter': 1000, 'random_state': None, 'tol': 0.001}

На основе переборе сформированы следующие оптимальные значения гиперпараметров:

    - 'C': 0.001, 
    - 'max_iter': 1000, 
    - 'random_state': None, 
    - 'tol': 0.001
    
Используем их на обучающей выборке вместе с гиперпараметрами `solver` и `penalty`, показавшими наилучший результат.

In [17]:
model_logreg = LogisticRegression(random_state=None, C=0.001, solver='lbfgs',max_iter=1000, tol=0.001)
model_logreg.fit(features_train, target_train)
result_train_logreg = model_logreg.score(features_train, target_train)
print(f'Величина метрики качества accurancy на валидационной выборке равна: {result_train_logreg}') 

Величина метрики качества accurancy на валидационной выборке равна: 0.7142116182572614


In [18]:
model_logreg = LogisticRegression(random_state=None, C=0.001, solver='newton-cg', penalty='none', max_iter=1000, tol=0.001)
model_logreg.fit(features_train, target_train)
result_train_logreg = model_logreg.score(features_train, target_train)
print(f'Величина метрики качества accurancy на валидационной выборке равна: {result_train_logreg}') 

Величина метрики качества accurancy на валидационной выборке равна: 0.7531120331950207


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


Увы, также и тут результат получается подобный. Значение качества модели при использовании перебора `GridSearchCV` на требуемую величину в 0.75 влияет слабо.

### Сравнения с другими моделями классификации

Ввиду "натянутого" результата работы модели `логистической регрессии` само собой напрашивается применение иных моделей по оговорённому выше плану.

#### Исследование модели классификации деревом решений


In [91]:
# создание объекта модели классификации деревом решений
model_tree_cl = DecisionTreeClassifier(random_state=12345)

# обучение модели 
model_tree_cl.fit(features_train, target_train)

# предсказания значения целевой переменной для обучающей выборки
train_predictions = model_tree_cl.predict(features_train)

# предсказания значения целевой переменной для валидационной выборки
valid_predictions = model_tree_cl.predict(features_valid)

# предсказания значения целевой переменной для тестовой выборки
test_predictions = model_tree_cl.predict(features_test)

In [92]:
print('Величина метрики качества accurancy на обучающей выборке равна:', accuracy_score(target_train, train_predictions))
print('Величина метрики качества accurancy на валидационной выборке равна:', accuracy_score(target_valid, valid_predictions))
print('Величина метрики качества accurancy на тестовой выборке равна:', accuracy_score(target_test, test_predictions))

Величина метрики качества accurancy на обучающей выборке равна: 1.0
Величина метрики качества accurancy на валидационной выборке равна: 0.713841368584759
Величина метрики качества accurancy на тестовой выборке равна: 0.7309486780715396


Если принять, как данность значение `accurancy`, равно 1, что подразумевает отсутствие ошибок при обучении модели
Налицо случай переобучения. Ввиду того, что переобучение характерно для моделей с большой величиной глубины дерева, необходимо провести изменить соответствующие гиперпараметры. Рассматриваемые гиперпараметры можно вывести командой `model_tree_cl.get_params()`, а их описание взять из [официальной документации](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier):

- `criterion` - функция для измерения качества разбиения
- `splitter` - стратегия выбора разбиения в каждом узле
- `max_depth` - максимальная глубина дерева

- `min_samples_split=2` - минимальное количество экземпляров, которое может содержаться в узле для дальнейшего разбиения
- `min_samples_leaf=1` - минимальное количество экземпляров, которое может содержаться в листе
- `min_weight_fraction_leaf=0.0` - минимальная взвешенная часть суммы всех весов входных экземпляров, необходимых в листовом узле
- `max_features` - максимальное количество фичей, которые рассматриваются при выборе лучшего разбиения
- `random_state` - начальное состояние генератора случайных чисел
- `max_leaf_nodes` - максимальное количество листьев
- `min_impurity_decrease` - минимальное уменьшение неоднородности
- `class_weight` - вес класса в виде
- `ccp_alpha` - сложность стоимости

Оптимальные значения гиперпараметров для подбора можно взять из [документации Kaggle](https://www.kaggle.com/code/zohrab/rus-sklearn-decisiontreeclassifier-hyperparams), поскольку в ней расписано достаточно ясно. Используем первые три.

Посчитаем значение `accurancy` при заданных трёх значениях гиперпараметров. Сравним предсказанное в ходе обучения на тренировочной выборке значение доли правильных ответов и отношение числа правильных ответов к тестовой выборке. Сведём всё в одну таблицу, поскольку значений у нас всего должно получиться 28, а таблица более проста в чтении для такого числа строк.

In [83]:
result_train_tree_cl = pd.DataFrame(columns=['Качество разбиения выборки criterion', 'Стратегия выбора разбиения выборки splitter', 'Глубина дерева обучения max_depth', 'Величина метрики качества accurancy на обучающей выборке', 'Величина метрики качества accurancy на тестовой выборке'])

criterion_types = ['gini', 'entropy']
splitter_types = ['best', 'random']

for criterion_type in criterion_types:
    for depth in range(1,8):
        for splitter_type in splitter_types:
            try:
                model_tree_cl = DecisionTreeClassifier(random_state=12345, criterion=criterion_type, splitter=splitter_type, max_depth=depth)
                model_tree_cl.fit(features_train, target_train)
                train_predictions = model_tree_cl.predict(features_train)
                accurancy_train_tree_cl = accuracy_score(target_train, train_predictions)
                test_predictions = model_tree_cl.predict(features_test)
                accurancy_test_tree_cl = accuracy_score(target_test, test_predictions)
                result_train_tree_cl = result_train_tree_cl.append({'Величина метрики качества accurancy на обучающей выборке': accurancy_train_tree_cl,
                                                                    'Величина метрики качества accurancy на тестовой выборке': accurancy_test_tree_cl,
                                                                    'Качество разбиения выборки criterion': criterion_type, 
                                                                    'Стратегия выбора разбиения выборки splitter': splitter_type, 
                                                                    'Глубина дерева обучения max_depth': depth}, ignore_index=True
                                                                  )
            except:
                print(f'Алгоритм дерева решений не смог найти оптимальное значение параметра accurancy') 

In [84]:
display(result_train_tree_cl.sort_values('Величина метрики качества accurancy на обучающей выборке', ascending=False))

,Качество разбиения выборки criterion,Стратегия выбора разбиения выборки splitter,Глубина дерева обучения max_depth,Величина метрики качества accurancy на обучающей выборке,Величина метрики качества accurancy на тестовой выборке
12,gini,best,7,0.855809,0.799378
26,entropy,best,7,0.848029,0.796267
10,gini,best,6,0.837656,0.776050
24,entropy,best,6,0.833506,0.780715
8,gini,best,5,0.820021,0.783826
22,entropy,best,5,0.817427,0.785381
6,gini,best,4,0.810685,0.774495
20,entropy,best,4,0.808091,0.777605
4,gini,best,3,0.807573,0.779160
18,entropy,best,3,0.807573,0.779160


Мы оказались посрамлены, когда при [принятии решения сделали ставку на модель _логистической регрессии_ в пользу перед моделью _дерева решений_](#Исследование-модели). На деле _дерево решений_ куда как с меньшей ошибкой выдаёт результат. 

В таблице выше данные отсортированы по величине метрики качества на обучающей выборке. Невооружённым глазом видно, как при увеличении глубины дерева обучения ошибка обучения снижается. Однако `max_depth` далеко не единственный гиперпараметр, вляющий на улучшение метрики качества. 

Также нам видится важным гиперпараметром `splatter`. Наибольшие значения метрики качества демонстрируются при использовании значения `best` стратегии выбора разбиения выборки. При идентичных значениях `max_depth` метрика качества выше именно у `best`. А вот гиперпараметр качества разбиения выборки показывает примерно одинаковые результаты в системе с двумя другими гиперпараметрами. Т.е. прямой зависимости между значениями `criterion` и числом ошибок обучения выборки не обнаружено.

Достичь необходимый результат в 0.75 возможно уже при следующих гиперпараметрах: max_depth=`4` и splitter=`random`. Однако чем выше величина `accurancy` на обучающей выборке, тем заметнее становится видна более высокая точность на обучающей выборке, нежели на выборке тестовой. Т.е. явный случай переобучения (хотя в случае для глубины дерева в единицу и вовсе характерно недообучение). С другой стороны, в зависимости от величины датасета отличие в 5-10% между значениями `accurancy` на разных выборках может быть и допустимым. У нас же стоит чёткая цель - доведение числа верных ответов хотя бы до 0.75. Значит, оперируя текущими результатами, можно сделать вывод о достижении цели при значениях основных гиперпараметров, выделив из них следующие:

|max_depth|criterion|splitter|
|-|-|-|
|4|entropy/gini|random|
|6|entropy/gini|random|
|5|entropy/gini|random|

#### Исследование модели классификации - случайный лес

В принципе поставленная цель может считаться достигнутой, но, поскольку у нас стоит задача исследования модели квалификации `случайный лес`, то необходимо её выполнить

In [101]:
# создание объекта модели классификации случайный лес
model_ran_for = RandomForestClassifier(random_state=12345, n_estimators=3)

# обучение модели 
model_ran_for.fit(features_train, target_train)

# предсказания значения целевой переменной для обучающей выборки
train_predictions = model_ran_for.predict(features_train)

# предсказания значения целевой переменной для валидационной выборки
valid_predictions = model_ran_for.predict(features_valid)

# предсказания значения целевой переменной для тестовой выборки
test_predictions = model_ran_for.predict(features_test)

print('Величина метрики качества accurancy на обучающей выборке равна:', accuracy_score(target_train, train_predictions))
print('Величина метрики качества accurancy на валидационной выборке равна:', accuracy_score(target_valid, valid_predictions))
print('Величина метрики качества accurancy на тестовой выборке равна:', accuracy_score(target_test, test_predictions))

Величина метрики качества accurancy на обучающей выборке равна: 0.9507261410788381
Величина метрики качества accurancy на валидационной выборке равна: 0.7387247278382582
Величина метрики качества accurancy на тестовой выборке равна: 0.7573872472783826


Вновь, как и в случае с `деревом решений` при гиперпараметрах, выставленных по-умолчанию (как в теории _ЯП_), значения метрики качества на обучающей выборке гораздо выше, нежели на выборках валидационной и тестовой. Перечислим гиперпараметры, которыми будем оперировать, и наиболее применимые их значения:

- `n_estimators` - количество деревьев в лесу (чаще всего принимает значения от 10 до 100)
- `max_depth` - максимальная глубина дерева (чаще всего принимает значения от 5 до 20)
- `min_samples_split` - минимальное количество наблюдений, необходимое для того, чтобы узел мог быть разделен (чаще всего принимает значения от 2 до 10)
- `min_samples_leaf` - минимальное количество наблюдений в листе дерева (чаще всего принимает значения от 1 до 5)
- `max_features` - количество признаков, используемых для поиска наилучшего разделения (чаще всего принимает значения от sqrt(features) до features).
- `criterion` - функция для измерения качества разбиения

Используем первые два параметра.

In [149]:
result_train_ran_for = pd.DataFrame(columns=['Количество деревьев в лесу n_estimators', 'Глубина дерева обучения max_depth', 'accurancy_train_ran_for', 'accurancy_test_ran_for'])

for depth in range(1,21,5):
    for est in range(1,101,10):
        try:
            model_ran_for = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
            model_ran_for.fit(features_train, target_train)
            train_predictions = model_ran_for.predict(features_train)
            accurancy_train_ran_for = accuracy_score(target_train, train_predictions)
            test_predictions = model_ran_for.predict(features_test)
            accurancy_test_ran_for = accuracy_score(target_test, test_predictions)
            result_train_ran_for = result_train_ran_for.append(
                {
                     'accurancy_train_ran_for': accurancy_train_ran_for,
                     'accurancy_test_ran_for': accurancy_test_ran_for,
                     'Количество деревьев в лесу n_estimators': est,
                     'Глубина дерева обучения max_depth': depth
                }, 
                ignore_index=True
            )
        except:
            print(f'Алгоритм дерева решений не смог найти оптимальное значение параметра accurancy') 

In [138]:
# при желании можно раскомментировать строки с кодом, но мы их посчитали для промежуточной оценки, 
# поэтому чтобы не загружать проектную работу, не станем выводить лишнюю информацию
# display(result_train_ran_for)
# result_train_ran_for[(result_train_ran_for['accurancy_test_ran_for'] >= 0.80)]

In [151]:
#pd.set_option('display.max_rows', None)
result_train_ran_for[
    (result_train_ran_for['accurancy_test_ran_for'] >= 0.75) &
    (result_train_ran_for['accurancy_test_ran_for'] <= 0.77) |
    (result_train_ran_for['accurancy_train_ran_for'] >= 0.75) &
    (result_train_ran_for['accurancy_train_ran_for'] <= 0.77)
].sort_values('accurancy_train_ran_for', ascending=False)

,Количество деревьев в лесу n_estimators,Глубина дерева обучения max_depth,accurancy_train_ran_for,accurancy_test_ran_for
30,1.0,16.0,0.882780,0.768274
3,31.0,1.0,0.781120,0.763608
4,41.0,1.0,0.767116,0.755832
6,61.0,1.0,0.764523,0.744946
8,81.0,1.0,0.764523,0.744946
5,51.0,1.0,0.764004,0.744946
7,71.0,1.0,0.764004,0.744946
9,91.0,1.0,0.761929,0.743390
2,21.0,1.0,0.751556,0.741835


В случае использования модели квалификации `случайный лес` редко, когда критерий качества для тестовой выборки достигает 0.80. В таких случаях разрыв между значениями `accurancy` для обучающей и тестовой выборками становится существенным. 

Прежде всего, действительно нужно [подтвердить описанный выше факт](#Исследование-модели) - данная модель действительно самая медленная. Неслучайно для оценки ревьюера поставил шаг для гиперпараметра `n_estimators` в 10 и с диапазоном глубин деревье от 1 до 20. Однако изначально модель была обучена для количества _деревьев в лесу_ от 1 до 100 с шагом 2, но такой расчёт оказался крайне длительным.

Наилучший результат, удовлетворяющий цели проектной работы, достигается при 27, 29 и 30 `деревьях` и `глубине` в 1 дерево. Всё же, как и предсказывалось, для выборки с малым числом признаков модель случайного леса не является оптимальной.

### Проверка модели на адекватность (бонус) 

<span style="color:green">
Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.
<span>

Пока к задаче не приступал, но 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
